In [11]:
import os
import keras
import cv2
import h5py
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
from tensorflow.keras import regularizers
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop, Adam
import numpy as np
import random
from   tensorflow.keras.preprocessing.image import img_to_array, load_img
import os
from glob import glob
from keras.models import Model
from keras.layers import Input, Dense, GlobalMaxPool2D, GlobalAvgPool2D, Concatenate, Multiply, Dropout, Subtract
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

In [12]:
print(os.listdir("../input/urecamain/Train"))

['Non-Fire', 'Fire']


In [13]:
# # detect and init the TPU
# tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)

# # instantiate a distribution strategy
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)


In [14]:
#with tpu_strategy.scope():
model = tf.keras.models.Sequential([

tf.keras.layers.Conv2D(16, (3,3), activation='relu',padding = 'same', input_shape=(240, 240, 3)),
#tf.keras.layers.MaxPooling2D(2, 2),

tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
tf.keras.layers.MaxPooling2D(2,2),

tf.keras.layers.Flatten(),

tf.keras.layers.Dense(1024, activation='relu'),
tf.keras.layers.Dropout(0.01),
tf.keras.layers.Dense(512, activation='relu',kernel_regularizer=regularizers.l2(0.01)),
tf.keras.layers.Dropout(0.01),
#tf.keras.layers.Dense(128, activation='relu'),
tf.keras.layers.Dense(32, activation='relu'),

tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',
          optimizer=Adam(lr=0.001),
          metrics=['accuracy'])



In [15]:
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 240, 240, 16)      448       
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 238, 238, 32)      4640      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 119, 119, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 117, 117, 64)      18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 58, 58, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 56, 56, 64)        36928     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 28, 28, 64)       

In [16]:
# from kaggle_datasets import KaggleDatasets
# #print(KaggleDatasets())
# GCS_PATH = KaggleDatasets().get_gcs_path() # you can list the bucket with "!gsutil ls $GCS_PATH"

In [17]:
# # On Kaggle you can also use KaggleDatasets().get_gcs_path() to obtain the GCS path of a Kaggle dataset
# filenames = tf.io.gfile.glob(GCS_PATH) # list files on GCS
# dataset = tf.data.TFRecordDataset(filenames)
# # dataset = dataset.map(...) # TFRecord decoding here...


In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
        rescale=1./255,
        horizontal_flip=True)
vali_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory(
        '../input/urecamain/Train',
        target_size=(240, 240),
        batch_size=128,
        classes = ['Fire','Non-Fire'],
        shuffle = True,
        
        class_mode='binary')
validation_generator = vali_datagen.flow_from_directory(
        '../input/urecamain/Vali',
        target_size=(240, 240),
        batch_size=64,
        classes = ['Fire','Non-Fire'],
        shuffle = True,
        class_mode='binary')
test_generator = test_datagen.flow_from_directory(
        '../input/urecamain/Test',
        target_size=(240, 240),
        batch_size=64,
        classes = ['Fire','Non-Fire'],
        shuffle = True,
        class_mode='binary')

# train_generator = train_datagen.flow_from_directory(
#         '../input/urecasegfire/Train',
#         target_size=(240, 240),  
#         batch_size=128, 

#         class_mode='binary')

Found 6003 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [19]:
file_path = "./Final1.h5"
checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
reduce_on_plateau = ReduceLROnPlateau(monitor="loss", mode="min", factor=0.1, patience=5, verbose=1)
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=8)
callbacks_list = [ checkpoint, reduce_on_plateau, es]

history = model.fit(
      train_generator,
      steps_per_epoch=46, 
      callbacks=callbacks_list,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=31,
      verbose=1)

Train for 46 steps, validate for 31 steps
Epoch 1/100
45/46 [============================>.] - ETA: 0s - loss: 3.3628 - accuracy: 0.6308
Epoch 00001: val_accuracy improved from -inf to 0.58115, saving model to ./Final1.h5
46/46 [==============================] - 36s 772ms/step - loss: 3.3132 - accuracy: 0.6329 - val_loss: 1.1967 - val_accuracy: 0.5811
Epoch 2/100
45/46 [============================>.] - ETA: 0s - loss: 0.6810 - accuracy: 0.7611
Epoch 00002: val_accuracy improved from 0.58115 to 0.74143, saving model to ./Final1.h5
46/46 [==============================] - 35s 754ms/step - loss: 0.6791 - accuracy: 0.7615 - val_loss: 0.6079 - val_accuracy: 0.7414
Epoch 3/100
45/46 [============================>.] - ETA: 0s - loss: 0.4277 - accuracy: 0.8321
Epoch 00003: val_accuracy improved from 0.74143 to 0.90121, saving model to ./Final1.h5
46/46 [==============================] - 34s 742ms/step - loss: 0.4254 - accuracy: 0.8332 - val_loss: 0.3058 - val_accuracy: 0.9012
Epoch 4/100
45/4

Epoch 29/100
45/46 [============================>.] - ETA: 0s - loss: 0.0186 - accuracy: 0.9970
Epoch 00029: val_accuracy improved from 0.94960 to 0.95010, saving model to ./Final1.h5
46/46 [==============================] - 34s 730ms/step - loss: 0.0186 - accuracy: 0.9971 - val_loss: 0.2524 - val_accuracy: 0.9501
Epoch 30/100
45/46 [============================>.] - ETA: 0s - loss: 0.0176 - accuracy: 0.9969
Epoch 00030: val_accuracy did not improve from 0.95010
46/46 [==============================] - 34s 741ms/step - loss: 0.0180 - accuracy: 0.9966 - val_loss: 0.2965 - val_accuracy: 0.9430
Epoch 31/100
45/46 [============================>.] - ETA: 0s - loss: 0.0192 - accuracy: 0.9969
Epoch 00031: val_accuracy did not improve from 0.95010
46/46 [==============================] - 33s 728ms/step - loss: 0.0191 - accuracy: 0.9969 - val_loss: 0.2923 - val_accuracy: 0.9415
Epoch 32/100
45/46 [============================>.] - ETA: 0s - loss: 0.0207 - accuracy: 0.9965
Epoch 00032: val_accur

In [20]:
# evaluate model
model.load_weights('./Final1.h5')
lossandacc = model.evaluate_generator(test_generator,verbose=1)
print(lossandacc)

OSError: Unable to open file (unable to open file: name = './Final.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [ ]:
from IPython.display import FileLink
FileLink('./Final1.h5')